### Mixtral 8x7B - Mixture of Experts

#### RAG with LlamaIndex - Nvidia CUDA + WSL (Windows Subsystem for Linux) + Word documents + Local LLM

### NOTE: YOU WILL NEED LLAMA-CPP-PYTHON VERSION v0.2.23 OR HIGHER<br>RUN: pip install --upgrade llama_cpp_python

This notebook demonstrates the use of LlamaIndex for Retrieval Augmented Generation using Windows' WSL and an Nvidia's CUDA.

Environment:
- Windows 11
- Anaconda environment
- Nvidia RTX 3090
- LLMs - Mistral 7B, Llama 2 13B Chat, Orca 2, Mixtral 8x7B etc. - Quantized versions

Your Data:
- Add Word documents to the "Data" folder for the RAG to use

Package versions:
- See the "conda_package_versions.txt" for the full list of versions in the conda environment (generated using "conda list").
- See/utilise the "requirements.txt" file (note that you need to have installed the CUDA Toolkit using the instructions below, the versions are very important).

Local LLMs:
- I downloaded the quantized versions of the LLMs from huggingface.co - thanks to TheBloke who provided these quantized GGUF models. You can use higher quantized versions or different LLMs - just be aware that LLMs may have different prompt templates so be sure to use the correct prompt template format (e.g. Llama 2 requires a specific format for best results - see the Llama code for a function that creates the prompt).
- See the main README for links to all the models that I downloaded

Important libraries to "pip install":
- llama-cpp-python
- transformers
- llama-index
- docx2txt
- sentence-transformers

Notes:
Getting the Nvidia CUDA libraries installed correctly for use within WSL was challenging, I followed the steps from these two links:

1. CUDA Toolkit version 12.3 (latest as of 2023-11-03)
- https://docs.nvidia.com/cuda/wsl-user-guide/index.html
2. Install instructions within WSL:
- https://developer.nvidia.com/cuda-downloads?target_os=Linux&target_arch=x86_64&Distribution=WSL-Ubuntu&target_version=2.0&target_type=deb_local

To tell if you are utilising your Nvidia graphics card, in your command prompt, while in the conda environment, type "nvidia-smi". You should see your graphics card and you're notebook is running you should see your utilisation increase.

#### 1. Prepare Llama Index for use

In [1]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext

#### 2. Load the Word document(s)

Note: A fictitious story about Thundertooth a dinosaur who has travelled to the future. Thanks ChatGPT!

In [2]:
documents = SimpleDirectoryReader("./Data/").load_data()

#### 3. Instantiate the model

In [3]:
import torch

from llama_index.llms import LlamaCPP
from llama_index.llms.llama_utils import messages_to_prompt, completion_to_prompt
llm = LlamaCPP(
    model_url=None, # We'll load locally.
    model_path='./Models/mixtral-8x7b-instruct-v0.1.Q4_K_M.gguf', # 4-bit model
    temperature=0.1,
    max_new_tokens=1024, # Increasing to support longer responses
    context_window=8192, # Mistral7B has an 8K context-window
    generate_kwargs={},
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": 40}, # 40 was a good amount of layers for the RTX 3090, you may need to decrease yours if you have less VRAM than 24GB
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True
)

llama_model_loader: loaded meta data with 26 key-value pairs and 995 tensors from ./Models/mixtral-8x7b-instruct-v0.1.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q4_K     [  4096, 32000,     1,     1 ]
llama_model_loader: - tensor    1:          blk.0.ffn_gate.0.weight q4_K     [  4096, 14336,     1,     1 ]
llama_model_loader: - tensor    2:          blk.0.ffn_down.0.weight q4_K     [ 14336,  4096,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_up.0.weight q4_K     [  4096, 14336,     1,     1 ]
llama_model_loader: - tensor    4:          blk.0.ffn_gate.1.weight q4_K     [  4096, 14336,     1,     1 ]
llama_model_loader: - tensor    5:          blk.0.ffn_down.1.weight q4_K     [ 14336,  4096,     1,     1 ]
llama_model_loader: - tensor    6:            blk.0.ffn_up.1.weight q4_K     [  4096, 14336,     1,     1 ]
llama_model_loader: - tensor    7:          blk.0.ffn_gate.2.weight q4_K     [  4096, 1

#### 4. Checkpoint

Are you running on GPU? The above output should include near the top something like:
> ggml_init_cublas: found 1 CUDA devices:

And in the full text near the bottom should be:
> llm_load_tensors: using CUDA for GPU acceleration

#### 5. Embeddings

Convert your source document text into embeddings.

The embedding model is from huggingface, this one performs well.

> https://huggingface.co/thenlper/gte-large


In [4]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import LangchainEmbedding, ServiceContext

embed_model = LangchainEmbedding(
  HuggingFaceEmbeddings(model_name="thenlper/gte-large")
)

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: thenlper/gte-large
Load pretrained SentenceTransformer: thenlper/gte-large
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cuda
Use pytorch device: cuda


#### 6. Prompt Template

Prompt template for Mistral:

>     <s>[INST] {prompt} [/INST]

#### 7. Service Context

For chunking the document into tokens using the embedding model and our LLM

In [5]:
service_context = ServiceContext.from_defaults(
    chunk_size=256, # Number of tokens in each chunk
    llm=llm,
    embed_model=embed_model
)

#### 8. Index documents

In [6]:
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

#### 9. Query Engine

Create a query engine, specifying how many citations we want to get back from the searched text (in this case 3).

The DB_DOC_ID_KEY is used to get back the filename of the original document

In [7]:
from llama_index.query_engine import CitationQueryEngine
query_engine = CitationQueryEngine.from_args(
    index,
    similarity_top_k=3,
    # here we can control how granular citation sources are, the default is 512
    citation_chunk_size=256,
)

# For citations we get the document info
DB_DOC_ID_KEY = "db_document_id"

#### 10. Prompt and Response function

Pass in a question, get a response back.

IMPORTANT: The prompt is in the question, adjust it to match what you want the LLM to act like and do.

In [8]:
def RunQuestion(questionText):
    queryQuestion = "<s>[INST] You are a technology specialist. Answer questions in a positive, helpful and empathetic way. Answer the following question: " + questionText + " [/INST]"

    response = query_engine.query(queryQuestion)

    return response

#### 11. Questions to test with

In [9]:
TestQuestions = [
    "Summarise the story for me",
    "Who was the main protagonist?",
    "Did they have any children? If so, what were their names?",
    "Did anything eventful happen?",
]

#### 12. Run Questions through model (this can take a while) and see citations

Runs each test question, saves it to a dictionary for output in the last step.

Note: Citations are the source documents used and the text the response is based on. This is important for RAG so you can reference these documents for the user, and to ensure it's utilising the right documents.

In [10]:
qa_pairs = []

for index, question in enumerate(TestQuestions, start=1):
    question = question.strip() # Clean up

    print(f"\n{index}/{len(TestQuestions)}: {question}")

    response = RunQuestion(question) # Query and get  response

    qa_pairs.append((question.strip(), str(response).strip())) # Add to our output array

    # Displays the citations
    for index, node in enumerate(response.source_nodes, start=1):
        print(f"{index}/{len(response.source_nodes)}: |{node.node.metadata['file_name']}| {node.node.get_text()}")

    # Uncomment the following line if you want to test just the first question
    # break 


1/4: Summarise the story for me


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

1/3: |Thundertooth Part 3.docx| Source 1:
Thundertooth



One fateful day, as the citizens of the futuristic city went about their daily lives, a collective gasp echoed through the streets as a massive meteor hurtled towards Earth. Panic spread like wildfire as people looked to the sky in horror, realizing the impending catastrophe. The city's advanced technology detected the threat, and an emergency broadcast echoed through the streets, urging everyone to seek shelter.



Thundertooth, ever the protector of his newfound home, wasted no time. With a determined gleam in his eyes, he gathered his family and hurried to the city's command center, where Mayor Grace and the leading scientists were coordinating the evacuation efforts.



The mayor, recognizing Thundertooth's intelligence and resourcefulness, approached him. "Thundertooth, we need a plan to divert or neutralize the meteor. Our technology can only do so much, but with your unique abilities, perhaps we can find a solution."



T


llama_print_timings:        load time =  138066.93 ms
llama_print_timings:      sample time =      58.40 ms /   195 runs   (    0.30 ms per token,  3338.87 tokens per second)
llama_print_timings: prompt eval time =  177431.40 ms /  1062 tokens (  167.07 ms per token,     5.99 tokens per second)
llama_print_timings:        eval time =   59389.35 ms /   194 runs   (  306.13 ms per token,     3.27 tokens per second)
llama_print_timings:       total time =  237338.16 ms


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...


llama_print_timings:        load time =  138066.93 ms
llama_print_timings:      sample time =      51.35 ms /   176 runs   (    0.29 ms per token,  3427.53 tokens per second)
llama_print_timings: prompt eval time =   57190.57 ms /   838 tokens (   68.25 ms per token,    14.65 tokens per second)
llama_print_timings:        eval time =   52135.84 ms /   175 runs   (  297.92 ms per token,     3.36 tokens per second)
llama_print_timings:       total time =  109723.31 ms


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Llama.generate: prefix-match hit


1/3: |Thundertooth Part 3.docx| Source 1:
Thundertooth



One fateful day, as the citizens of the futuristic city went about their daily lives, a collective gasp echoed through the streets as a massive meteor hurtled towards Earth. Panic spread like wildfire as people looked to the sky in horror, realizing the impending catastrophe. The city's advanced technology detected the threat, and an emergency broadcast echoed through the streets, urging everyone to seek shelter.



Thundertooth, ever the protector of his newfound home, wasted no time. With a determined gleam in his eyes, he gathered his family and hurried to the city's command center, where Mayor Grace and the leading scientists were coordinating the evacuation efforts.



The mayor, recognizing Thundertooth's intelligence and resourcefulness, approached him. "Thundertooth, we need a plan to divert or neutralize the meteor. Our technology can only do so much, but with your unique abilities, perhaps we can find a solution."



T


llama_print_timings:        load time =  138066.93 ms
llama_print_timings:      sample time =      57.71 ms /   193 runs   (    0.30 ms per token,  3344.13 tokens per second)
llama_print_timings: prompt eval time =   62441.21 ms /   913 tokens (   68.39 ms per token,    14.62 tokens per second)
llama_print_timings:        eval time =   57481.40 ms /   192 runs   (  299.38 ms per token,     3.34 tokens per second)
llama_print_timings:       total time =  120385.59 ms


#### 13. Output responses

In [11]:
for index, (question, answer) in enumerate(qa_pairs, start=1):
    print(f"{index}/{len(qa_pairs)} {question}\n\n{answer}\n\n--------\n")

1/4 Summarise the story for me

Thundertooth, a talking dinosaur, stumbles upon a futuristic city where he finds acceptance and a new home [2][1]. Initially, he struggles to find food that satisfies his hunger without causing harm to the city's inhabitants. However, with the help of Mayor Grace, they discover a sustainable solution [3]. Thundertooth, inspired by the advanced technology around him, starts a toy factory that becomes a sensation, producing magical widgets that delight children and adults alike [2]. Eventually, he meets Seraphina, and they start a family together [2]. When a massive meteor threatens Earth, Thundertooth steps up to help, working with the city's leading scientists to find a solution to divert or neutralize the meteor [1]. Throughout his journey, Thundertooth becomes a symbol of unity between the past and future, teaching everyone about understanding and cooperation.

--------

2/4 Who was the main protagonist?

The main protagonist in this story is Thunderto